This notebook was inpired by this LlamaIndex notebook:

https://colab.research.google.com/drive/1c5ORIlqs3YMWosDSMgs6_ZHb5eiANS1c?usp=sharing

Making some changes to it with the only intention of trying ideas and learning.

Notice that I am assuming you have the relevant API_KEYs as environmental variables.

In [2]:
from typing import Optional

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
from llama_index.core.bridge.pydantic import BaseModel
import random
import string

Things to know:
- We will define an Agent that manages bookings for a restaurant.
    - We will use a `Booking` Pydantic class to track and represent a booking.
    - We will have the next methods:
        - `get_booking_state`: Use booking ID to retrieve the state of a booking
        - `update_booking`: Update the booking by modifying a specific property.
        - `create_booking`: Initialize a booking and requesting more info.
        - `confirm_booking`: Complete the booking by making sure all information has been provided

## Booking Class

In [38]:
# we will store booking under random IDs
bookings = {}

class Booking(BaseModel):
    name: Optional[str] = None
    email: Optional[str] = None
    phone: Optional[str] = None
    date: Optional[str] = None
    time: Optional[str] = None


def get_booking_state(user_id: str) -> str:
    """Get the current state of a booking for a given booking ID."""
    try:
        return str(bookings[user_id].dict())
    except:
        return f"Booking ID {user_id} not found"


def update_booking(user_id: str, property: str, value: str) -> str:
    """Update a property of a booking for a given booking ID. Only enter details that are explicitly provided."""
    booking = bookings[user_id]
    setattr(booking, property, value)
    return f"Booking ID {user_id} updated with {property} = {value}"


def create_booking() -> str:
    """Create a new booking and return the booking ID."""
    # Generate the random ID
    user_id = ''.join(random.choice(string.ascii_uppercase) for _ in range(6))
    bookings[user_id] = Booking()
    return f"Booking created with id {user_id}, but not yet confirmed. Please provide your name, email, phone, date, and time."


def confirm_booking(user_id: str) -> str:
    """Confirm a booking for a given booking ID."""
    booking = bookings[user_id]

    if booking.name is None:
        raise ValueError("Please provide your name.")

    if booking.email is None:
        raise ValueError("Please provide your email.")

    if booking.phone is None:
        raise ValueError("Please provide your phone number.")

    if booking.date is None:
        raise ValueError("Please provide the date of your booking.")

    if booking.time is None:
        raise ValueError("Please provide the time of your booking.")

    return f"Booking for Name: {booking.name} and ID: {user_id} is confirmed!"

In [39]:
get_booking_state_tool = FunctionTool.from_defaults(fn=get_booking_state, return_direct=True)
update_booking_tool = FunctionTool.from_defaults(fn=update_booking)
create_booking_tool = FunctionTool.from_defaults(fn=create_booking, return_direct=True)
confirm_booking_tool = FunctionTool.from_defaults(fn=confirm_booking, return_direct=True)

tools = [
    get_booking_state_tool,
    update_booking_tool,
    create_booking_tool,
    confirm_booking_tool,
]

In [40]:
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"You are now connected to the booking system and helping a user with making a booking. "
            "Only enter details that the user has explicitly provided. "
            "Do not make up any details."
        ),
    )
]

In [43]:
worker = FunctionCallingAgentWorker(
    tools=tools,
    llm=OpenAI(model='gpt-3.5-turbo'),
    prefix_messages=prefix_messages,
    max_function_calls=10,
    allow_parallel_tool_calls=False,
    verbose=True,
)
agent = AgentRunner(worker)

In [44]:
response = agent.chat("Hello! I would like to make a booking.")

Added user message to memory: Hello! I would like to make a booking.
=== Calling Function ===
Calling function: create_booking with args: {}
=== Function Output ===
Booking created with id UDHTNJ, but not yet confirmed. Please provide your name, email, phone, date, and time.


In [45]:
response = agent.chat("Sure! My name is bubl-ai, and my email is contact.bubl.ai@gmail.com")

Added user message to memory: Sure! My name is bubl-ai, and my email is contact.bubl.ai@gmail.com
=== Calling Function ===
Calling function: update_booking with args: {"property": "name", "user_id": "UDHTNJ", "value": "bubl-ai"}
=== Function Output ===
Booking ID UDHTNJ updated with name = bubl-ai
=== Calling Function ===
Calling function: update_booking with args: {"property": "email", "user_id": "UDHTNJ", "value": "contact.bubl.ai@gmail.com"}
=== Function Output ===
Booking ID UDHTNJ updated with email = contact.bubl.ai@gmail.com
=== LLM Response ===
Thank you for providing your name and email. Could you please provide your phone number, preferred date, and time for the booking?


In [46]:
response = agent.chat("My phone number is 111-222-3456, reservation for this sunday at 10 am")

Added user message to memory: My phone number is 111-222-3456, reservation for this sunday at 10 am
=== Calling Function ===
Calling function: update_booking with args: {"property": "phone", "user_id": "UDHTNJ", "value": "111-222-3456"}
=== Function Output ===
Booking ID UDHTNJ updated with phone = 111-222-3456
=== Calling Function ===
Calling function: update_booking with args: {"property": "date", "user_id": "UDHTNJ", "value": "this sunday"}
=== Function Output ===
Booking ID UDHTNJ updated with date = this sunday
=== Calling Function ===
Calling function: update_booking with args: {"property": "time", "user_id": "UDHTNJ", "value": "10 am"}
=== Function Output ===
Booking ID UDHTNJ updated with time = 10 am
=== LLM Response ===
Thank you for providing all the details. Your booking is almost complete. Would you like to confirm the booking now?


In [47]:
response = agent.chat("yes")

Added user message to memory: yes
=== Calling Function ===
Calling function: confirm_booking with args: {"user_id": "UDHTNJ"}
=== Function Output ===
Booking for Name: bubl-ai and ID: UDHTNJ is confirmed!


In [48]:
response = agent.chat("provide the booking details")

Added user message to memory: provide the booking details
=== Calling Function ===
Calling function: get_booking_state with args: {"user_id": "UDHTNJ"}
=== Function Output ===
{'name': 'bubl-ai', 'email': 'contact.bubl.ai@gmail.com', 'phone': '111-222-3456', 'date': 'this sunday', 'time': '10 am'}


In [49]:
response = agent.chat("provide the booking details for ID QWEE")

Added user message to memory: provide the booking details for ID QWEE
=== Calling Function ===
Calling function: get_booking_state with args: {"user_id": "QWEE"}
=== Function Output ===
Booking ID QWEE not found


In [50]:
response = agent.chat("What user ID exist on your data?")

Added user message to memory: What user ID exist on your data?
=== LLM Response ===
The user ID "UDHTNJ" exists in the data. If you have any other user IDs you would like to inquire about, please provide them.
